# 🌏 Sagaing Fault Plate Reconstruction

This notebook creates an animated visualization of the **Sagaing Fault's** tectonic evolution from 50 Ma to present day.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/sagaing-fault-reconstruction/blob/main/sagaing_fault_reconstruction.ipynb)

## About the Sagaing Fault

| Parameter | Value |
|-----------|-------|
| **Length** | ~1,400 km |
| **Type** | Right-lateral strike-slip |
| **Formation** | ~22-15 Ma (Miocene) |
| **Slip Rate** | 18-24 mm/yr |
| **Total Displacement** | 330-450 km |

**Author:** Tin Ko Oo, Mahidol University, Thailand

## 1. Install Dependencies

Run this cell first to install required packages.

In [ ]:
# Install required packages
!pip install gplately cartopy -q

# Install ffmpeg for video encoding
!apt-get install ffmpeg -qq

print("✅ Installation complete!")

## 2. Import Libraries

In [ ]:
import gplately
from gplately import PlateReconstruction, PlotTopologies
import pygplates
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D
from IPython.display import HTML, Video
import warnings
warnings.filterwarnings("ignore")

print(f"✅ GPlately version: {gplately.__version__}")

## 3. Define Sagaing Fault Trace

Coordinates compiled from Sloan et al. (2017), USGS, and GEM Global Active Faults Database.

In [ ]:
# Sagaing Fault trace coordinates (lon, lat)
# From Gulf of Martaban to Eastern Himalayan Syntaxis

SAGAING_FAULT_TRACE = [
    # Southern Section - Gulf of Martaban to Bago
    (96.30, 16.50),  # Offshore termination
    (96.35, 17.00),  # Bago segment
    (96.25, 17.50),
    
    # Central Section - Naypyidaw to Mandalay
    (96.10, 18.00),  # Pyu segment
    (96.00, 18.50),
    (95.95, 19.00),  # Naypyidaw segment
    (95.90, 19.50),
    (96.00, 20.00),  # Meiktila segment
    (96.05, 20.50),
    (96.10, 21.00),
    (96.00, 21.50),  # Sagaing segment (type locality)
    (95.95, 22.00),
    (96.00, 22.50),
    
    # Northern Section - Horsetail zone
    (96.10, 23.00),  # Tawma segment
    (96.20, 23.50),
    (96.35, 24.00),  # Ban Mauk segment
    (96.50, 24.50),
    (96.70, 25.00),
    (97.00, 25.50),  # Northern splay zone
    (97.30, 26.00),
    (97.50, 26.50),  # Approaches Eastern Syntaxis
]

print(f"✅ Fault trace defined with {len(SAGAING_FAULT_TRACE)} points")
print(f"   Latitude range: {SAGAING_FAULT_TRACE[0][1]}°N to {SAGAING_FAULT_TRACE[-1][1]}°N")

## 4. Configuration

In [ ]:
# ============================================
# CONFIGURATION - Modify as needed
# ============================================

CONFIG = {
    # Time settings (Ma = millions of years ago)
    'time_start': 50,           # Before India-Eurasia collision peak
    'time_end': 0,              # Present day
    'time_step': 1,             # Time per frame
    'fault_initiation': 22,     # When Sagaing Fault formed
    
    # Animation settings
    'fps': 10,
    'dpi': 150,
    'output_file': 'sagaing_fault_reconstruction.mp4',
    
    # Map extent [lon_min, lon_max, lat_min, lat_max]
    'extent': [92, 102, 10, 28],  # Myanmar focus
    
    # Visualization
    'figure_size': (14, 10),
    'fault_color': '#FF0000',
    'fault_width': 3.0,
}

n_frames = int((CONFIG['time_start'] - CONFIG['time_end']) / CONFIG['time_step']) + 1

print(f"📊 Configuration:")
print(f"   Time range: {CONFIG['time_start']} Ma → {CONFIG['time_end']} Ma")
print(f"   Fault initiation: {CONFIG['fault_initiation']} Ma")
print(f"   Total frames: {n_frames}")
print(f"   Output: {CONFIG['output_file']}")

## 5. Load Plate Model

In [ ]:
print("📥 Loading Müller et al. (2019) plate model...")
print("   (This may take 1-2 minutes on first run)\n")

# Download plate model
gdownload = gplately.DataServer("Muller2019")
rotation_model, topology_features, static_polygons = gdownload.get_plate_reconstruction_files()
coastlines, continents, COBs = gdownload.get_topology_geometries()

# Create reconstruction model
model = PlateReconstruction(rotation_model, topology_features, static_polygons)
gplot = PlotTopologies(model, coastlines=coastlines, continents=continents, COBs=COBs, time=0)

print("✅ Plate model loaded successfully!")

## 6. Create Fault Feature

In [ ]:
def create_sagaing_fault_feature(coords, plate_id=807, valid_time_start=22):
    """Create pygplates feature for Sagaing Fault."""
    points = [(lat, lon) for lon, lat in coords]
    fault_polyline = pygplates.PolylineOnSphere(points)
    
    fault_feature = pygplates.Feature()
    fault_feature.set_geometry(fault_polyline)
    fault_feature.set_reconstruction_plate_id(plate_id)
    fault_feature.set_valid_time(valid_time_start, 0)
    fault_feature.set_name("Sagaing Fault")
    
    return fault_feature

fault_feature = create_sagaing_fault_feature(SAGAING_FAULT_TRACE)
print(f"✅ Sagaing Fault feature created")
print(f"   Plate ID: 807 (Shan/Sibumasu)")
print(f"   Valid time: 22 Ma → 0 Ma")

## 7. Helper Functions

In [ ]:
def reconstruct_fault(fault_feature, rotation_model, reconstruction_time):
    """Reconstruct fault position at given time."""
    valid_time = fault_feature.get_valid_time()
    if valid_time and reconstruction_time > valid_time[0]:
        return None
    
    reconstructed_geometries = []
    pygplates.reconstruct(fault_feature, rotation_model, 
                          reconstructed_geometries, reconstruction_time)
    
    if not reconstructed_geometries:
        return None
    
    coords = []
    for recon_geom in reconstructed_geometries:
        geom = recon_geom.get_reconstructed_geometry()
        for point in geom.get_points():
            lat, lon = point.to_lat_lon()
            coords.append((lon, lat))
    return coords


def calculate_displacement(reconstruction_time, slip_rate=18, 
                           max_displacement=400, initiation_time=22):
    """Calculate cumulative fault displacement."""
    if reconstruction_time > initiation_time:
        return 0.0
    duration_years = (initiation_time - reconstruction_time) * 1e6
    displacement_km = duration_years * slip_rate * 1e-6
    return min(displacement_km, max_displacement)


print("✅ Helper functions defined")

## 8. Preview Single Time Slice

In [ ]:
# Preview at specific time
preview_time = 10  # Ma

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
ax.set_extent(CONFIG['extent'], crs=ccrs.PlateCarree())

# Gridlines
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5)
gl.top_labels = False
gl.right_labels = False

# Update time
gplot.time = preview_time

# Plot layers
try:
    gplot.plot_continents(ax, facecolor='#E8DCC4', edgecolor='none', alpha=0.7)
    gplot.plot_coastlines(ax, color='#4A4A4A', linewidth=0.8)
    gplot.plot_ridges(ax, color='blue', linewidth=1.5)
    gplot.plot_trenches(ax, color='purple', linewidth=1.5)
except:
    pass

# Plot fault
fault_coords = reconstruct_fault(fault_feature, rotation_model, preview_time)
if fault_coords:
    lons = [c[0] for c in fault_coords]
    lats = [c[1] for c in fault_coords]
    ax.plot(lons, lats, 'r-', linewidth=3, transform=ccrs.PlateCarree(), label='Sagaing Fault')

# Annotations
disp = calculate_displacement(preview_time)
ax.set_title(f'Sagaing Fault Reconstruction: {preview_time} Ma', fontsize=14, fontweight='bold')
ax.text(0.98, 0.02, f'Cumulative slip: ~{disp:.0f} km', transform=ax.transAxes,
        fontsize=10, ha='right', va='bottom',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"\n🗺️ Preview at {preview_time} Ma")

## 9. Create Animation

⚠️ **Note:** This may take 5-15 minutes depending on settings.

In [ ]:
print(f"🎬 Creating animation with {n_frames} frames...")
print(f"   Estimated time: {n_frames * 3}-{n_frames * 8} seconds\n")

fig = plt.figure(figsize=CONFIG['figure_size'])
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())

def update(frame):
    reconstruction_time = CONFIG['time_start'] - frame * CONFIG['time_step']
    if reconstruction_time < CONFIG['time_end']:
        reconstruction_time = CONFIG['time_end']
    
    ax.clear()
    ax.set_extent(CONFIG['extent'], crs=ccrs.PlateCarree())
    
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5)
    gl.top_labels = False
    gl.right_labels = False
    
    gplot.time = reconstruction_time
    
    try:
        gplot.plot_continents(ax, facecolor='#E8DCC4', edgecolor='none', alpha=0.7)
        gplot.plot_coastlines(ax, color='#4A4A4A', linewidth=0.8)
        gplot.plot_ridges(ax, color='blue', linewidth=1.5)
        gplot.plot_trenches(ax, color='purple', linewidth=1.5)
    except:
        pass
    
    # Plot fault
    fault_coords = reconstruct_fault(fault_feature, rotation_model, reconstruction_time)
    if fault_coords:
        lons = [c[0] for c in fault_coords]
        lats = [c[1] for c in fault_coords]
        ax.plot(lons, lats, color=CONFIG['fault_color'], 
                linewidth=CONFIG['fault_width'], transform=ccrs.PlateCarree())
    
    # Title and time
    ax.set_title('Sagaing Fault Tectonic Evolution', fontsize=14, fontweight='bold')
    
    time_str = f'{reconstruction_time:.0f} Ma'
    if reconstruction_time <= CONFIG['fault_initiation']:
        time_str += ' (Fault active)'
    ax.text(0.02, 0.98, time_str, transform=ax.transAxes, fontsize=14,
            va='top', ha='left', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.9))
    
    # Displacement
    disp = calculate_displacement(reconstruction_time)
    if disp > 0:
        ax.text(0.98, 0.02, f'Cumulative slip: ~{disp:.0f} km', transform=ax.transAxes,
                fontsize=10, ha='right', va='bottom',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Legend
    legend_elements = [
        Line2D([0], [0], color='#4A4A4A', linewidth=1, label='Coastlines'),
        Line2D([0], [0], color=CONFIG['fault_color'], linewidth=3, label='Sagaing Fault'),
        Line2D([0], [0], color='blue', linewidth=1.5, label='Spreading Ridge'),
        Line2D([0], [0], color='purple', linewidth=1.5, label='Subduction Zone'),
    ]
    ax.legend(handles=legend_elements, loc='lower left', fontsize=8)
    
    if frame % 10 == 0:
        print(f"   Frame {frame}/{n_frames}: {reconstruction_time:.0f} Ma")
    
    return []

anim = FuncAnimation(fig, update, frames=n_frames, interval=1000/CONFIG['fps'], blit=False)

print("\n💾 Saving animation...")
anim.save(CONFIG['output_file'], fps=CONFIG['fps'], dpi=CONFIG['dpi'], writer='ffmpeg')
print(f"\n✅ Animation saved as '{CONFIG['output_file']}'")

plt.close()

## 10. Display Animation

In [ ]:
Video(CONFIG['output_file'], embed=True, width=800)

## 11. Download Animation

In [ ]:
try:
    from google.colab import files
    files.download(CONFIG['output_file'])
    print(f"📥 Downloading {CONFIG['output_file']}...")
except:
    print(f"ℹ️ File saved locally as '{CONFIG['output_file']}'")

---

## 📚 References

1. Müller, R.D., et al. (2019). A global plate model including lithospheric deformation. *Tectonics*, 38.
2. Sloan, R.A., et al. (2017). Active tectonics of Myanmar and the Andaman Sea. *Geol. Soc. London Mem.*, 48.
3. Socquet, A., & Pubellier, M. (2005). Cenozoic deformation in western Yunnan. *Tectonophysics*, 391.
4. Curray, J.R. (2005). Tectonics and history of the Andaman Sea region. *J. Asian Earth Sci.*, 25.

---

<center><b>Made with ❤️ by Tin Ko Oo, Mahidol University</b></center>